<hr style="color:green" />
<h1 style="color:green">COSC2673 Assignment 2: Image Classification for Cancerous Cells</h1>
<h2 style="color:green">File 05a: PyTorch - Get Mean and Standard Deviation</h2>
<hr style="color:green" />

<p>
In a number of future experiments with PyTorch, The Tensor Data Loaders will make use of a Normalize operation on the image data, which requires the Means and Standard Deviations of the R, G and B numbers
</p>
<p>
Rather than loading all the training images then computing the mean and std in every experiment, run this file to get the mean and standard deviation values, which can then be hardcoded into other NN training experiments.
<p>
<p> 
Note, any time the images are reloaded and re-split into training, validation and test splits, this will need to be rerun to recalculate the values for the new training split
</p>

In [2]:
import pandas as pd
import numpy as np
import os
import cv2

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image


c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\nelso\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Configure this script as to whether it runs on Google Colab, or locally

In [3]:
# When on Google Colab, running full training, change both to true. Locally, advised set both to false
isGoogleColab = False
useFullData = False

In [4]:
# In local, the base directory is the current directory
baseDirectory = "./"

if isGoogleColab:
    from google.colab import drive
    
    # If this is running on Google colab, assume the notebook runs in a "COSC2673" folder, which also contains the data files 
    # in a subfolder called "image_classification_data"
    drive.mount("/content/drive")
    !ls /content/drive/'My Drive'/COSC2673/

    # Import the directory so that custom python libraries can be imported
    import sys
    sys.path.append("/content/drive/MyDrive/COSC2673/")

    # Set the base directory to the Google Drive specific folder
    baseDirectory = "/content/drive/MyDrive/COSC2673/"

Import the custom python files that contain reusable code

In [83]:
import data_basic_utility as dbutil
import graphing_utility as graphutil
import statistics_utility as statsutil

import a2_utility as a2util
import pytorch_utility as ptutil
from pytorch_utility import CancerBinaryDataset
from pytorch_utility import CancerCellTypeDataset


# randomSeed = dbutil.get_random_seed()
randomSeed = 266305
print("Random Seed: " + str(randomSeed))

Random Seed: 266305


In [84]:
# this file should have previously been created in the root directory
dfImages = pd.read_csv(baseDirectory + "images_main.csv")

In [85]:
# Get The training Split and the Validation Split
dfImagesTrain = dfImages[dfImages["trainValTest"] == 0].reset_index()
dfImagesVal = dfImages[dfImages["trainValTest"] == 1].reset_index()

dfImagesTrain.head()

,index,ImageName,isCancerous,cellType,trainValTest
0,0,./Image_classification_data/patch_images\1.png,0,0,0
1,1,./Image_classification_data/patch_images\10.png,0,0,0
2,3,./Image_classification_data/patch_images\1000.png,1,2,0
3,4,./Image_classification_data/patch_images\10000...,0,1,0
4,5,./Image_classification_data/patch_images\10001...,0,1,0


In [86]:
# Load the images into a numpy array. This is so we can find the mean and std of the RBG values in order to Normalize the images for the Neural Network
imgsTrain = []
for imageName in dfImagesTrain["ImageName"]:
    # In colab, we want to remove the leading "./" from the image name
    if isGoogleColab:
        if imageName.startswith("./"):
            imageName = imageName[2:]

    imageName = imageName.replace("\\", "/")
    img = cv2.imread(baseDirectory + imageName)
    imgsTrain.append(img)

    

In [87]:
print(len(imgsTrain))
print(imgsTrain[0])

7837
[[[213 187 220]
  [214 189 220]
  [240 215 246]
  ...
  [226 197 231]
  [207 175 210]
  [211 183 215]]

 [[209 182 216]
  [237 210 242]
  [254 228 255]
  ...
  [217 182 223]
  [216 178 220]
  [231 198 237]]

 [[226 197 232]
  [252 224 255]
  [239 211 245]
  ...
  [223 190 228]
  [222 187 227]
  [221 188 226]]

 ...

 [[203 151 212]
  [193 142 204]
  [206 154 217]
  ...
  [224 187 242]
  [230 191 247]
  [232 190 247]]

 [[217 162 222]
  [213 161 223]
  [205 153 216]
  ...
  [221 176 233]
  [219 174 233]
  [224 181 240]]

 [[219 161 217]
  [210 153 214]
  [223 167 230]
  ...
  [215 169 228]
  [213 169 228]
  [220 177 235]]]


In [88]:
# Define a transform to convert images to PyTorch tensors
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Convert images to tensors and concatenate them into a single tensor
tensor_images = torch.cat([transform(img).unsqueeze(0) for img in imgsTrain])

# Calculate the mean and standard deviation of the tensor values
tensor_train_mean = tensor_images.mean(dim=(0, 2, 3))
tensor_train_std = tensor_images.std(dim=(0, 2, 3))

print("Mean:", tensor_train_mean)
print("Standard deviation:", tensor_train_std)

print("\n\n")
print("Lines of code to copy over:")
print("train_mean = torch." + tensor_train_mean)
print("train_std = torch." + tensor_train_std)

Mean: tensor([0.8035, 0.5909, 0.7640])
Standard deviation: tensor([0.1246, 0.1947, 0.1714])


Take these lines of code and update them in pytorch_utility.py, which other experiment files will use in the Normalize Transform for Data Loaders. That way, if they change, it just needs to be updated in one location.